In [ ]:
# ライブラリの読み込み
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import itertools
import time
import re
import os
import glob
import MeCab
from numpy.random import *

pd.set_option("display.max_rows", 250)
pd.set_option("display.max_columns", 100)

# 文書のデータフレームを作成

## 分割済みデータから作成

In [ ]:
# データの読み込み
nlp_path = "D:/Statistics/data/NLP/"
tdb_path = "D:/Statistics\data/scenario_extract/tdb/sample/"
symbol_mapping = pd.read_csv(nlp_path + "dic/symbol_mapping.csv")
tdb = pd.read_csv(tdb_path + "sample_descriptions_by_sentence.csv", dtype="str")
tdb["INTERNAL_ID"] = tdb["INTERNAL_ID"].astype("int")
tdb["SENTENCE_ID"] = tdb["SENTENCE_ID"].astype("int")
tdb["JGNY_KNJ_2"] = tdb["JGNY_KNJ_2"].str.normalize("NFKC")
tdb["JGNY_KNJ_2"] = tdb["JGNY_KNJ_2"].str.replace(",", "")
tdb["JGNY_KNJ_2"] = tdb["JGNY_KNJ_2"].str.replace("および", "及び")
for i in range(symbol_mapping.shape[0]):
    tdb["JGNY_KNJ_2"] = tdb["JGNY_KNJ_2"].str.replace(symbol_mapping["symbol"].iloc[i], symbol_mapping["mapping"].iloc[i])

In [ ]:
# データフレームを出力
nlp_path = "D:/Statistics/data/NLP/"
result = pd.DataFrame({"KGCD": tdb["KGCD"], "d_id": tdb["SENTENCE_ID"], "pt": tdb["INTERNAL_ID"] , "text": tdb["JGNY_KNJ_2"]})
result.to_csv(nlp_path + "/tdb/tdb_result_by_sentence.csv", index=None)
result[["text"]].to_csv(nlp_path + "/tdb/tdb_text_by_sentence.txt", index=None, header=None)

## オリジナルデータから作成

In [ ]:
# 記号を出力
# temp = pd.read_csv("D:/Statistics/data/NLP/tdb/tdb_corpus.csv")
# kigo = pd.unique(temp["word"].iloc[np.where(temp["class"]=="記号")])
# target_kigo = kigo[np.array([len(kigo[i]) for i in range(len(kigo))]) > 1]
# target_mapping = np.array([" ".join(list(target_kigo[i])) for i in range(len(target_kigo))])
# output = pd.DataFrame({"symbol": target_kigo, "mapping": target_mapping})
# output.to_csv("D:/Statistics/data/NLP/dic/symbol_mapping_.csv")

In [ ]:
# データの読み込み
tdb = pd.read_csv(tdb_path + "sample_company_business_descriptions.csv", dtype=str)
tdb["JGNY_KNJ_2"] = tdb["JGNY_KNJ_2"].str.normalize("NFKC")
tdb["JGNY_KNJ_2"] = tdb["JGNY_KNJ_2"].str.replace(",", "")
tdb["JGNY_KNJ_2"] = tdb["JGNY_KNJ_2"].str.replace("および", "及び")

In [ ]:
# 文章を句読点で分割
# データの設定
kgcd = np.array(tdb["KGCD"])
text = np.array(tdb["JGNY_KNJ_2"])
D = len(kgcd)

# 検索条件を設定
brackets_start = np.array(["\(", "「"])
brackets_end = np.array(["\)", "」"])
condition1 = "(?!$)(?:[^(。]*" + brackets_start[0] + "[^)]*" + brackets_end[0] + ")*[^(。]*(?:。|$)"
condition2 = "(?!$)(?:[^(。]*" + brackets_start[1] + "[^)]*" + brackets_end[1] + ")*[^(。]*(?:。|$)"
condition = "(" + condition1 + "|" +  condition2 + ")"

# 文を文章に分割
sentence_list = []
kgcd_list = []
no_list1 = []
no_list2 = []
max_no = 0
for i in range(D):
    sentence_list.append(np.array(re.findall(condition, text[i])))
    kgcd_list.append(np.repeat(kgcd[i], len(sentence_list[i])))
    no_list1.append(np.arange(len(sentence_list[i])))
    no_list2.append(np.arange(len(sentence_list[i])) + max_no)
    max_no = np.max(no_list2[i]) + 1
    
# リストを変換
sentence = np.hstack((sentence_list))
kgcd_id = np.hstack((kgcd_list))
no1 = np.hstack((no_list1))
no2 = np.hstack((no_list2))
del sentence_list, kgcd_list, no_list1, no_list2

In [ ]:
# 括弧内の文書を除去する
# 括弧内を置き換える
new_sentence = pd.Series(np.repeat("", len(sentence)).astype("object"))
for i in range(len(sentence)):
    new_sentence[i] = re.sub("\(.+?\)", "", sentence[i])
    new_sentence[i] = re.sub("\[.+?\]", "", new_sentence[i])
    new_sentence[i] = re.sub("\〔.+?\〕", "", new_sentence[i])
    
# 残った括弧を除去
index = np.where(new_sentence.str.contains("[\(|\)|\[|\]|〔|〕]"))[0]
target_index = index[new_sentence.iloc[index].str.contains("\(\)")==False]
new_sentence.iloc[target_index] = new_sentence.iloc[target_index].str.replace("[\(|\)|\[|\]|〔|〕]", "")
# new_sentence = new_sentence.iloc[np.where(new_sentence.str.replace(" ", ""))[0]]

In [ ]:
# 記号の連続を置き換える
sentence_series = pd.Series(new_sentence)
for i in range(symbol_mapping.shape[0]):
    sentence_series = sentence_series.str.replace(symbol_mapping["symbol"].iloc[i], symbol_mapping["mapping"].iloc[i])
n = tdb.shape[0]

# マッピングされているかどうかチェック
for i in range(symbol_mapping.shape[0]):
    print([i, np.sum(sentence_series.str.contains(symbol_mapping["symbol"].iloc[i]))])

In [ ]:
# 確認用ファイルを出力
check = pd.DataFrame({"d_id": np.arange(len(sentence)), "text": sentence, "new_text": sentence_series,
                      "flag": np.array(sentence!=sentence_series, dtype="int")})
check.to_excel(nlp_path + "tdb/確認用.xlsx")

In [ ]:
# データフレームを出力
result = pd.DataFrame({"serial_no": np.arange(len(kgcd_id)), "kgcd": kgcd_id, "no": no1, "d_id": no2, "text": sentence_series})
result.to_csv(nlp_path + "/tdb/tdb_result_by_sentence.csv", index=None)
result[["text"]].to_csv(nlp_path + "/tdb/tdb_text_by_sentence.txt", index=None, header=None)

# 文節区切りに分割

In [ ]:
# 形態素解析を実行
# mecabの設定
N = len(sentence)
mecab_columns = ["word", "word_class", "class_detail1", "class_detail2", "class_detail3",
                 "inflectional1", "inflectional2", "genkei", "readings1", "readings2"]
mecab = MeCab.Tagger(" -d C:/Users/sana/dic/20190510_neologd")

# データの格納用配列
parsed_list = [i for i in range(N)]
kgcd_list = [i for i in range(N)]
no_list1 = [i for i in range(N)]
no_list2 = [i for i in range(N)]

# 文章ごとにMeCabを実行
for i in range(N):
    if i%1000==0:
        print(i)
    res = mecab.parse(sentence[i])
    parsed_split = pd.Series(res.split("\n")).str.split('\t|,').tolist()
    parsed_list[i] = pd.DataFrame.from_records(parsed_split[0:len(parsed_split)-2])
    parsed_list[i].columns = mecab_columns
    n = parsed_list[i].shape[0]

    # 単語ごとにidを付与
    kgcd_list[i] = np.repeat(kgcd_id[i], n)
    no_list1[i] = np.repeat(no1[i], n)
    no_list2[i] = np.repeat(no2[i], n)
    
# データフレームを定義
parsed_columns = ["word", "genkei", "word_class", "class_detail1", "class_detail2", "class_detail3",
                  "inflectional1", "inflectional2"]
kgcd_long = np.hstack((kgcd_list))
no_long1 = np.hstack((no_list1))
no_long2 = np.hstack((no_list2))
F = len(kgcd_long)
tdb_parsed1 = pd.DataFrame({"serial_no": np.arange(F),  "kgcd": kgcd_long, "no": no_long1, "d_id": no_long2})
tdb_parsed2 = pd.concat((parsed_list), axis=0)[parsed_columns]
tdb_parsed2.index = np.arange(tdb_parsed2.shape[0])
tdb_parsed = pd.concat((tdb_parsed1, tdb_parsed2), axis=1)
del kgcd_list, no_list1, no_list2, tdb_parsed1, tdb_parsed2

In [ ]:
# データフレームを出力
split = 3
path = "D:/Statistics/data/NLP/"
kgcd = np.unique(tdb_parsed["kgcd"])
split_kgcd = np.array_split(kgcd, split, 0)
for j in range(split):
    index = np.where(np.in1d(tdb_parsed["kgcd"], split_kgcd[j]))[0]
    tdb_split = tdb_parsed.iloc[index]
    tdb_split.to_excel(path + "tdb/tdb_parsed" + str(j) + ".xlsx")
del tdb_split